In [ ]:
suppressMessages(library('CellChat'))

In [ ]:
suppressMessages(library('tidyverse'))
suppressMessages(library('patchwork'))
suppressMessages(library('cowplot'))

suppressMessages(library('rhdf5'))
suppressMessages(library('Seurat'))

In [ ]:
suppressMessages(library(NMF))

In [ ]:
library(ggalluvial)

In [ ]:
suppressMessages(library(ComplexHeatmap))

In [ ]:
library("MetBrewer")

In [ ]:
suppressMessages(library(ggpubr))

In [ ]:
suppressMessages(library(future.apply))

In [ ]:
options(future.rng.onMisue = "ignore", future.seed = NULL)

In [ ]:
sessionInfo()

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=10,
        repr.plot.res = 200)

In [ ]:
source('function_single_cell_conversion.R')

In [ ]:
# cannot find the R/analysis.R file in the bin, so modified here
netClustering  <- function (object, slot.name = "netP", type = c("functional", 
    "structural"), comparison = NULL, k = NULL, methods = "kmeans", 
    do.plot = TRUE, fig.id = NULL, do.parallel = TRUE, nCores = 4, 
    k.eigen = NULL) 
{
    type <- match.arg(type)
    if (object@options$mode == "single") {
        comparison <- "single"
        cat("Classification learning of the signaling networks for a single dataset", 
            "\n")
    }
    else if (object@options$mode == "merged") {
        if (is.null(comparison)) {
            comparison <- 1:length(unique(object@meta$datasets))
        }
        cat("Classification learning of the signaling networks for datasets", 
            as.character(comparison), "\n")
    }
    comparison.name <- paste(comparison, collapse = "-")
    Y <- methods::slot(object, slot.name)$similarity[[type]]$dr[[comparison.name]]
    data.use <- Y
    if (methods == "kmeans") {
        if (!is.null(k)) {
            clusters = kmeans(data.use, k, nstart = 10)$cluster
        }
        else {
            N <- nrow(data.use)
            kRange <- seq(2, min(N - 1, 10), by = 1)
            if (do.parallel) {
                #future::plan("multiprocess", workers = nCores)
                future::plan("multisession", workers = 8)
                options(future.globals.maxSize = 1000 * 1024^2)
            }
            my.sapply <- ifelse(test = future::nbrOfWorkers() == 
                1, yes = pbapply::pbsapply, no = future.apply::future_sapply)
            results = my.sapply(X = 1:length(kRange), FUN = function(x) {
                idents <- kmeans(data.use, kRange[x], nstart = 10)$cluster
                clusIndex <- idents
                adjMat0 <- Matrix::Matrix(as.numeric(outer(clusIndex, 
                  clusIndex, FUN = "==")), nrow = N, ncol = N)
                return(list(adjMat = adjMat0, ncluster = length(unique(idents))))
            }, simplify = FALSE)
            adjMat <- lapply(results, "[[", 1)
            CM <- Reduce("+", adjMat)/length(kRange)
            res <- computeEigengap(as.matrix(CM))
            numCluster <- res$upper_bound
            clusters = kmeans(data.use, numCluster, nstart = 10)$cluster
            if (do.plot) {
                gg <- res$gg.obj
                ggsave(filename = paste0("estimationNumCluster_", 
                  fig.id, "_", type, "_dataset_", comparison.name, 
                  ".pdf"), plot = gg, width = 3.5, height = 3, 
                  units = "in", dpi = 300)
            }
        }
    }
    else if (methods == "spectral") {
        A <- as.matrix(data.use)
        D <- apply(A, 1, sum)
        L <- diag(D) - A
        L <- diag(D^-0.5) %*% L %*% diag(D^-0.5)
        evL <- eigen(L, symmetric = TRUE)
        plot(rev(evL$values)[1:30])
        Z <- evL$vectors[, (ncol(evL$vectors) - k.eigen + 1):ncol(evL$vectors)]
        clusters = kmeans(Z, k, nstart = 20)$cluster
    }
    if (!is.list(methods::slot(object, slot.name)$similarity[[type]]$group)) {
        methods::slot(object, slot.name)$similarity[[type]]$group <- NULL
    }
    methods::slot(object, slot.name)$similarity[[type]]$group[[comparison.name]] <- clusters
    return(object)
}

In [ ]:
# this is where the bug is, fixing not correct matrix sparcing and fixing the weird indents everywhere.......
# https://github.com/sqjin/CellChat/issues/210#issuecomment-1538673400

subset_CellChat_single_dataset_rev = function(object, cells.use = NULL, idents.use = NULL, 
                               group.by = NULL, thresh = 0.05){
    
    labels <- object@idents # factor without names
    names(labels) = colnames(object@data) # new line to name by cell id and fix the level.use is NULL issue
    level.use0 <- levels(labels)
    
    if (!is.null(cells.use)) {
        level.use <- levels(labels)[levels(labels) %in% unique(as.character(labels[cells.use]))] 
        cells.use.index <- which(names(labels) %in% cells.use)
    }
    
    if(!is.null(idents.use)) {
        level.use <- levels(labels)[levels(labels) %in% unique(as.character(idents.use))] 
        cells.use.index <- which(labels %in% level.use)
    }
        
    if (nrow(object@data) > 0) {
        data.subset <- object@data[, cells.use.index]
    } else {
        data.subset <- matrix(0, nrow = 0, ncol = 0)
    }
    if (nrow(object@data.project) > 0) {
        data.project.subset <- object@data.project[, cells.use.index]
    } else {
        data.project.subset <- matrix(0, nrow = 0, ncol = 0)
    }
    
    data.signaling.subset <- object@data.signaling[, cells.use.index]
        
    meta.subset <- object@meta[cells.use.index, , drop = FALSE]

    cat("Update slots object@images, object@net, object@netP in a single dataset...", '\n')

    group.existing <- level.use0[level.use0 %in% level.use]
    group.existing.index <- which(level.use0 %in% level.use)

    images <- object@images
    for (images.j in names(images)) {
        values <- images[[images.j]]

        if (images.j %in% c("coordinates")) {
            values.new <- values[cells.use.index, ]
            images[[images.j]] <- values.new
        }
        if (images.j %in% c("distance")) {
            values.new <- values[group.existing.index, group.existing.index, drop = FALSE]
            images[[images.j]] <- values.new
        }
    }
    images.subset <- images


    net <- object@net
    for (net.j in names(net)) {
      values <- net[[net.j]]
      if (net.j %in% c("prob","pval")) {
        values.new <- values[group.existing.index, group.existing.index, ,drop = FALSE] # this is the bug line
        net[[net.j]] <- values.new
      }
      if (net.j %in% c("count","sum","weight")) {
        values.new <- values[group.existing.index, group.existing.index, drop = FALSE]
        net[[net.j]] <- values.new
      }
    }
    net.subset <- net
    
    # summarise related ligands and receptors
    netP = computeCommunProbPathway(net = net.subset, pairLR.use = object@LR$LRsig, thresh = thresh)
    netP$centr = netAnalysis_computeCentrality(net = netP$prob) # netP not net.subset
    netP.subset <- netP
    idents.subset <- object@idents[cells.use.index]
    idents.subset <- factor(idents.subset, levels = level.use)
    
    
    object.subset <- methods::new(
    Class = "CellChat",
    data = data.subset,
    data.signaling = data.signaling.subset,
    data.project = data.project.subset,
    images = images.subset,
    net = net.subset,
    netP = netP.subset,
    meta = meta.subset,
    idents = idents.subset,
    var.features = object@var.features,
    LR = object@LR,
    DB = object@DB,
    options = object@options
    )
    return(object.subset)
    }

In [ ]:
plot_theme_tmod =  theme(title = element_text(size = 14),
                   axis.title = element_text(size = 14),
                   axis.text = element_text(size = 12),
                   legend.title =  element_text(size = 12),
                   text = element_text(size = 10),
                   plot.margin = margin(0,0,0,0,"cm")) 

In [ ]:
'%notin%' = Negate('%in%')

In [ ]:
data_folder = '/fast/users/twei_m/work/crc/datasets'

In [ ]:
new_data_folder = '/fast/users/twei_m/work/crc/datasets_new_preprocessing'

epi_cell_order = c('Stem','Stem_TA','Enterocyte_progenitor','Enterocytes','Immature_Goblet','Goblet','Tuft',
                                         'TC1','TC2','TC3','TC4')

In [ ]:
imm_str_cell_order = c('B_cells','Plasma_cells','CD4+_T_cells','CD8+_T_cells','Treg_cells',
                                         'Dendritic_cells','Macrophages','Mast_cells','Monocytes','Neutrophils',
                                         'Endothelial_cells','Fibroblasts','Pericytes')

In [ ]:
cell_type_order_with_Tumour_without_Monocytes = c('B_cells','Plasma_cells',
                                'CD4+_T_cells','CD8+_T_cells','Treg_cells',
                                'Dendritic_cells','Macrophages','Mast_cells',#'Monocytes',
                                                  'Neutrophils',
                                'Endothelial_cells','Fibroblasts','Pericytes',
                                'Tumour', 'Stem','Stem_TA','Tuft',
                                'Immature_Goblet','Goblet','Enterocyte_progenitor','Enterocytes'

                                )

In [ ]:
cell_type_order_with_Tumour_without_Monocytes_color = c('#7d8df0', '#AA82A7',
                                                        '#335B8E', '#79c9c9', '#c44545',
                                                        '#67C9FF', '#cf7708', '#B5B867', #  '#DEB058'
                                                        '#899DA4',
                                                         '#CE9486', '#8f7022','#1aa375',
                                                        '#AA0000', '#00CCFF', '#0072b1','#FF9900',
                                                        '#990099', '#660099','#999900', '#336600'
                                                        
)

In [ ]:
Numbat_and_Scitcem = read.csv('../datasets_new_preprocessing/20231108_Numbat_and_Scitcem_resolved_assignment_cellid.csv',
                             row.names=1)

In [ ]:
str_imm_id_tumour = read.csv('../datasets_new_preprocessing/str_and_imm_cellid_tumour.csv',
                             row.names=1)

In [ ]:
str_imm_id_normal = read.csv('../datasets_new_preprocessing/str_and_imm_cellid_normal.csv',
                             row.names=1)

### Read h5 and transform into Seurat

In [ ]:
h5_path = '../datasets_new_preprocessing/202307_adata_all_full_cell_type_annotation.h5'

In [ ]:
adata_h5 = H5Fopen(h5_path)

In [ ]:
# main matrix
adata_all = read_scanpy_h5(h5_path,
                           if_pca = TRUE, if_umap = TRUE, if_CB_counts=TRUE, if_raw_counts=FALSE)

In [ ]:
adata_all

In [ ]:
# get sample label
cat = c(adata_h5$obs$'sample'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'sample'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'sample')

In [ ]:
# get patient 
cat = c(adata_h5$obs$'patient'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'patient'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'patient')

In [ ]:
# get sample_origin 
cat = c(adata_h5$obs$'sample_origin'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'sample_origin'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'sample_origin')

In [ ]:
# get MS_status
cat = c(adata_h5$obs$'MS_status'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'MS_status'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'MS_status')

In [ ]:
# get cell type 1a 
cat = c(adata_h5$obs$'celltype_1a'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'celltype_1a'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'celltype_1a')

In [ ]:
# get cell type 2 
cat = c(adata_h5$obs$'cell_type_level2'$categories)
cat_obs = data.frame(cat_name = cat[apply(adata_h5$obs$'cell_type_level2'$codes, 1, function(x)x+1)],
                                              row.names = adata_h5$obs$`_index`)

adata_all = AddMetaData(object = adata_all, metadata = cat_obs, col.name = 'cell_type_level2')


In [ ]:
h5closeAll()

In [ ]:
# cell type in Uhlitz 
cell_type_list = c(unique(adata_all@meta.data['cell_type_level2'])$cell_type_level2) %>% sort()
cell_type_list = gsub('/', '_', cell_type_list)
cell_type_list = gsub(' ', '_', cell_type_list)
cell_type_list

In [ ]:
# replace the / cuz it will cuz a file saving problem
adata_all@meta.data  = adata_all@meta.data %>% mutate(cell_type_level2 = gsub('/', '_', cell_type_level2)) %>%
mutate(cell_type_level2 = gsub(' ', '_', cell_type_level2))

In [ ]:
adata_all

adata_all@meta.data = adata_all@meta.data %>% rownames_to_column() %>%
left_join(data.frame(patient = sort(unique(adata_all@meta.data$patient)),
           anatomy = c('0_Cecum', '0_Cecum', '7_Sigmoid', '2_Ascending', '2_Ascending',
                       '9_Rectum', '9_Unspecified', '6_Descending', '0_Cecum', '2_Ascending')),
          by = 'patient') %>%
column_to_rownames('rowname')

In [ ]:
colnames(adata_all@meta.data)

In [ ]:
table(adata_all@meta.data$sample_origin)

df = table(adata_all@meta.data[c('sample_origin', 'cell_type_level2')]) %>% as.data.frame() %>% 
group_by(cell_type_level2) %>%
mutate(Percentage = round(Freq/sum(Freq)*100,0),
       Sum = sum(Freq)) %>% 
group_by(sample_origin) %>% mutate(Percentage_sample = round(Freq/sum(Freq)*100,0),
       Sum_sample = sum(Freq))

df$cell_type_level2 = factor(df$cell_type_level2, levels = cell_type_order)

options(repr.plot.width=14, 
        repr.plot.height=6,
        repr.plot.res = 200)

df %>%
ggplot(aes(x = cell_type_level2, y = sample_origin)) + 
geom_tile(aes(fill = Freq), color = 'black') +
geom_text(aes(label = paste0(Freq, '\n', 
                             Percentage, '%\n',
                             Percentage_sample, '%'))) +
scale_fill_gradient(low = "grey95", high = "red") +
plot_theme_tmod + 
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      rect = element_blank(),
      axis.ticks = element_blank(),
      #aspect.ratio=0.5,
      axis.text.x = element_text(angle = 90)) +
guides(fill = guide_legend(title = "Count")) +
labs(title = 'Number of cell in each group\n',
     x = '\nCell type (including TCs)',
     y = 'Sample Origin\n') +
scale_x_discrete(labels = paste0(levels(df$cell_type_level2), ' (',unique(df$Sum),')')) +
scale_y_discrete(labels = paste0(levels(df$sample_origin), '\n (',unique(df$Sum_sample),')'))

# barplot
cell_type_merge_genomics = adata_all@meta.data[c('sample_origin', 'celltype_1a')] %>% 
merge(Numbat_and_Scitcem, by = 0, all.x = TRUE) %>%
mutate(cell_type_merge = coalesce(cell_identity, celltype_1a)) 

cell_type_merge_genomics$cell_type_merge = factor(cell_type_merge_genomics$cell_type_merge, 
                                                  levels = c('imm', 'str',
                                                             'normal_sample', 'genomically_normal', 
                                                             'no confident assignment', 'genomically_tumour'))

adata_all@meta.data = adata_all@meta.data %>% 
rownames_to_column() %>%
left_join(cell_type_merge_genomics[c('cell_type_merge', 'Row.names')], by = c('rowname' = 'Row.names'), copy = TRUE) %>%
column_to_rownames('rowname')

options(repr.plot.width=6, 
        repr.plot.height=6,
        repr.plot.res = 200)

cell_type_merge_genomics%>%
ggplot() +
geom_bar(aes(x = sample_origin, fill = cell_type_merge)) +
theme_void() + 
plot_theme_tmod

### cell type composition per sample

table(adata_all@meta.data[c('sample', 'celltype_1a')])

# epi, imm, str
cell_type_1a_per_sample = adata_all@meta.data[c('sample', 'sample_origin', 'celltype_1a')] %>%
mutate(sample_name = substr(sample, 1,5)) %>%
group_by(sample, celltype_1a) %>%
mutate(sum=n()) %>%
group_by(sample) %>%
mutate(sum_sample = n()) %>%
distinct() %>% 
arrange(desc(sample))

reverse_sample_order = fct_rev(unique(cell_type_1a_per_sample$sample))

options(repr.plot.width=7, 
        repr.plot.height=4,
        repr.plot.res = 200)

cell_type_1a_per_sample %>%
ggplot(aes(x = sum, y = factor(sample, levels(reverse_sample_order)), fill = celltype_1a)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#144FAC',
                             '#DD6B80',
                             '#4EAC57'),
                 name = 'Cell type',
                 labels = c('Epithelial ', 'Immune ', 'Stromal ')) +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(labels = paste0(levels(reverse_sample_order), ' (', 
                                 unique(cell_type_1a_per_sample$sum_sample), ')'))

options(repr.plot.width=7, 
        repr.plot.height=4,
        repr.plot.res = 200)

cell_type_1a_per_sample %>%
ggplot(aes(x = sum, y = sample_name, fill = celltype_1a)) +
geom_col(width = 0.8, position = 'stack', stat = 'identity') +
scale_fill_manual(values = c('#144FAC',
                             '#DD6B80',
                             '#4EAC57'),
                 name = 'Cell type',
                 labels = c('Epithelial ', 'Immune ', 'Stromal ')) +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') 

rev_sample_name = fct_rev(unique(cell_type_1a_per_sample$sample_name))

options(repr.plot.width=7, 
        repr.plot.height=4,
        repr.plot.res = 200)

cell_type_1a_per_sample %>%
ggplot(aes(x = sum, y = factor(sample_name, levels(rev_sample_name)) , fill = celltype_1a)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#144FAC',
                             '#DD6B80',
                             '#4EAC57'),
                 name = 'Cell type',
                 labels = c('Epithelial ', 'Immune ', 'Stromal ')) +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') 

# epi, imm, str, Numbat Scitcem
cell_type_merge_genomics_per_sample = adata_all@meta.data[c('sample', 'sample_origin', 'celltype_1a')] %>%
filter(sample_origin == 'tumour') %>%
merge(Numbat_and_Scitcem, by = 0, all.x = TRUE) %>%
mutate(cell_type_merge = coalesce(cell_identity, celltype_1a)) %>%
group_by(sample, cell_type_merge) %>%
mutate(sum=n()) %>%
group_by(sample) %>%
mutate(sum_sample = n()) %>%
distinct(sample,cell_type_merge, sum, sum_sample)

cell_type_merge_genomics_per_sample$cell_type_merge = factor(cell_type_merge_genomics_per_sample$cell_type_merge, 
                                                  levels = c('imm', 'str',
                                                             'normal_sample', 'genomically_normal', 
                                                             'no confident assignment', 'genomically_tumour'))

number_of_cells_per_sample = adata_all@meta.data %>% group_by(sample) %>% 
filter(sample_origin == 'tumour') %>%
summarise(sum=n()) %>%
arrange(desc(sample))

head(cell_type_merge_genomics_per_sample)

options(repr.plot.width=7, 
        repr.plot.height=3,
        repr.plot.res = 200)

cell_type_merge_genomics_per_sample %>%
ggplot(aes(x = sum, y = factor(sample, number_of_cells_per_sample$sample), fill = cell_type_merge)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
#geom_text(size = 3, position = position_stack(vjust = 0.5)) +
scale_fill_manual(values = c('#DD6B80',
                            '#4EAC57',
                            '#1f77b4',
                            '#9b1ee3',
                            '#ff7f0e'),
                   name = 'Cell identity\n(tumour samples)',
                   labels = c('Immune',
                             'Stromal', 
                             'Genomically normal',
                             'No confident assignment',
                              'Genomically tumour')) +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(labels = paste0(number_of_cells_per_sample$sample, ' (', number_of_cells_per_sample$sum, ')')) 

# only epi
options(repr.plot.width=7, 
        repr.plot.height=3,
        repr.plot.res = 200)

cell_type_merge_genomics_per_sample %>%
filter(!cell_type_merge %in% c('imm', 'str')) %>%
ggplot(aes(x = sum, y = factor(sample, number_of_cells_per_sample$sample), fill = cell_type_merge)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
#geom_text(size = 3, position = position_stack(vjust = 0.5)) +
scale_fill_manual(values = c(#'#DD6B80',
                            #'#4EAC57',
                            '#1f77b4',
                            '#9b1ee3',
                            '#ff7f0e'),
                   name = 'Cell identity\n(tumour samples)',
                   labels = c(#'Immune',
                             #'Stromal', 
                             'Genomically normal',
                             'No confident assignment',
                              'Genomically tumour')) +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(labels = paste0(number_of_cells_per_sample$sample, ' (', number_of_cells_per_sample$sum, ')')) 

#### cell type annotation

In [ ]:
adata_all@meta.data = adata_all@meta.data %>% 
merge(Numbat_and_Scitcem, by = 0, all.x = TRUE)  %>% # this will do rownames to column
mutate(cell_type_cell_identity = case_when(is.na(cell_identity) ~ cell_type_level2,
                                           cell_identity == 'normal_sample' ~ cell_type_level2,
                                           .default = paste0(cell_type_level2, '_', cell_identity))) %>%
column_to_rownames('Row.names')

# all epi in GN and NN
epi_cell_type_merge_genomics_per_sample = adata_all@meta.data[c('patient','sample', 'sample_origin', 
                                                                'cell_type_level2', 'cell_identity')] %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>% 
group_by(sample, cell_identity, cell_type_level2) %>%
mutate(sum=n()) %>%
group_by(sample) %>%
mutate(sum_sample = n()) %>%
distinct() %>%
arrange(desc(sample))


options(repr.plot.width=10, 
        repr.plot.height=3.5,
        repr.plot.res = 200)

epi_cell_type_merge_genomics_per_sample %>%
ggplot(aes(x = sum, y = patient, fill = cell_type_level2)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#999900', '#336600', '#660099', '#990099', '#00CCFF', '#0072b1',
       '#AA0000', '#FF0000', '#FF0099', '#EB9999', '#FF9900'),
                 name = 'Epithelial cell type')+
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Patient\n') +
facet_grid(cols = vars(cell_identity))



### differentiation cell states are preserved in genomically normal 

cell_state_across_CI= adata_all@meta.data[c('cell_identity', 'cell_type_level2', 'celltype_1a')] %>%
group_by(cell_identity, cell_type_level2) %>%
mutate(sum=n()) %>% distinct()

# NT NN
options(repr.plot.width=7, 
        repr.plot.height=2,
        repr.plot.res = 200)

cell_state_across_CI %>%
filter(!celltype_1a %in% c('imm', 'str')) %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
ggplot(aes(x = sum, y = cell_identity, fill = cell_type_level2)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90),
     legend.position = 'none') +
labs(x = '\nProportion\n', 
     y = 'Cell identity\n') +
scale_fill_manual(values = c('#999900', '#336600', '#660099', '#990099', '#00CCFF', '#0072b1',
       '#AA0000', '#FF0000', '#FF0099', '#EB9999', '#FF9900'),
                 name = 'Epithelial cell type')+
scale_y_discrete(labels = c('genomically normal','normal sample'))#+
#guides(fill=guide_legend(ncol=2))
#scale_y_discrete(labels = paste0(number_of_cells_per_sample$sample, ' (', number_of_cells_per_sample$sum, ')')) 

options(repr.plot.width=7, 
        repr.plot.height=3,
        repr.plot.res = 200)

cell_state_across_CI %>%
filter(!celltype_1a %in% c('imm', 'str')) %>%
ggplot(aes(x = sum, y = cell_identity, fill = cell_type_level2)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme_tmod +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Cell identity\n') +
scale_fill_manual(values = c('#999900', '#336600', '#660099', '#990099', '#00CCFF', '#0072b1',
       '#AA0000', '#FF0000', '#FF0099', '#EB9999', '#FF9900'),
                 name = 'Epithelial cell type')+
guides(fill=guide_legend(ncol=2))
#scale_y_discrete(labels = paste0(number_of_cells_per_sample$sample, ' (', number_of_cells_per_sample$sum, ')')) 

### Cellchat label-free mode: did not find the function .....

help(createCellChat)

### label-based mode

### Create a cell type / group label that can later separate NT and TT

adata_all@meta.data = adata_all@meta.data %>% 
merge(Numbat_and_Scitcem, by = 0, all.x = TRUE)  %>% # this will do rownames to column
mutate(cell_type_cell_identity = case_when(is.na(Numbat_and_Scitcem) ~ cell_type_level2,
                                           Numbat_and_Scitcem == 'normal_sample' ~ cell_type_level2,
                                           .default = paste0(cell_type_level2, '_', Numbat_and_Scitcem))) %>%
column_to_rownames('Row.names')

In [ ]:
# all genomically tumour as tumour
adata_all@meta.data = adata_all@meta.data %>% 
mutate(cell_identity_resolved_tumour = case_when(is.na(cell_identity) ~ cell_type_level2,
                                                 cell_identity == 'genomically_tumour' ~ 'Tumour',
                                                 .default = cell_type_level2))
                                                 #cell_identity == 'normal_sample' ~ cell_type_level2,

                                                 #.default = paste0(cell_type_level2, '_', cell_identity)))


In [ ]:
table(adata_all@meta.data['cell_identity_resolved_tumour'])

In [ ]:
# subset tumour samples vs normal sample
adata_GT_TME = subset(adata_all, subset = sample_origin == 'tumour' & 
                      cell_identity %in% c(NA, 'genomically_tumour') &
                      cell_type_level2 != 'Monocytes')

adata_GN_noTCs = subset(adata_all, subset = sample_origin == 'tumour' & cell_identity == 'genomically_normal' &
                 cell_type_level2 %notin% c('TC1', 'TC2', 'TC3', 'TC4', 'Monocytes'))

adata_n_noTCs = subset(adata_all, subset = sample_origin == 'normal' & 
                       cell_type_level2 %notin% c('TC1', 'TC2', 'TC3', 'TC4', 'Monocytes'))

In [ ]:
adata_all@meta.data = adata_all@meta.data %>% mutate(cell_id = rownames(adata_all@meta.data))

In [ ]:
most_DC2_tumour_like_cell_from_normal = subset(adata_all, subset = cell_id == 'p007n:CCTATCGAGATGAACT')

In [ ]:
most_DC2_tumour_like_cell_from_normal@meta.data['cell_type_level2'] = 'Tumour'

most_DC2_tumour_like_cell_from_normal@meta.data['cell_type_cell_identity'] = 'Tumour'

most_DC2_tumour_like_cell_from_normal@meta.data['cell_identity_resolved_tumour'] = 'Tumour'

In [ ]:
most_DC2_tumour_like_cell_from_normal@meta.data

In [ ]:
adata_GT_TME

In [ ]:
adata_GN_noTCs

In [ ]:
adata_n_noTCs

In [ ]:
adata_n = merge(adata_n_noTCs,most_DC2_tumour_like_cell_from_normal)

In [ ]:
adata_t = merge(adata_GT_TME, adata_GN_noTCs)

#### make cellchat obj

In [ ]:
cellchat_tumour = createCellChat(object = adata_t, group.by = "cell_identity_resolved_tumour")

In [ ]:
cellchat_normal = createCellChat(object = adata_n, group.by = "cell_identity_resolved_tumour")

In [ ]:
cellchat_normal@idents = factor(cellchat_normal@idents,  
                                     levels = cell_type_order_with_Tumour_without_Monocytes)

In [ ]:
cellchat_tumour@idents = factor(cellchat_tumour@idents,  
                                     levels = cell_type_order_with_Tumour_without_Monocytes)

In [ ]:
CellChatDB <- CellChatDB.human

In [ ]:
showDatabaseCategory(CellChatDB)

In [ ]:
# use all pathways 
CellChatDB.use <- CellChatDB
cellchat_normal@DB <- CellChatDB.use
cellchat_tumour@DB <- CellChatDB.use

In [ ]:
# filter to only genes exist in the database
cellchat_normal <- subsetData(cellchat_normal) 
cellchat_tumour <- subsetData(cellchat_tumour)

In [ ]:
plan("multisession", workers = 8)
# Error: Strategy 'multiprocess' is defunct in future (>= 1.32.0) [2023-03-06]. 
# Instead, explicitly specify either 'multisession' (recommended) or 'multicore'.

In [ ]:
options(future.globals.maxSize = 8000 * 1024^2)

In [ ]:
cellchat_normal <- identifyOverExpressedGenes(cellchat_normal)
cellchat_normal <- identifyOverExpressedInteractions(cellchat_normal)

In [ ]:
cellchat_tumour <- identifyOverExpressedGenes(cellchat_tumour)
cellchat_tumour <- identifyOverExpressedInteractions(cellchat_tumour)

In [ ]:
# project gene expression data onto PPI (Optional: when running it, USER should set `raw.use = FALSE` 
# in the function `computeCommunProb()` in order to use the projected data)
# cellchat <- projectData(cellchat, PPI.human)

triMean is used for calculating the average gene expression per cell group. 

Error in computeCommunProb(cellchat_normal): Please check `unique(object@idents)` and ensure that the factor levels are correct!
         You may need to drop unused levels using 'droplevels' function. e.g.,
         `meta$labels = droplevels(meta$labels, exclude = setdiff(levels(meta$labels),unique(meta$labels)))`
Traceback:

1. computeCommunProb(cellchat_normal)
2. stop("Please check `unique(object@idents)` and ensure that the factor levels are correct!\n         You may need to drop unused levels using 'droplevels' function. e.g.,\n         `meta$labels = droplevels(meta$labels, exclude = setdiff(levels(meta$labels),unique(meta$labels)))`")



In [ ]:
cellchat_normal <- computeCommunProb(cellchat_normal)

In [ ]:
cellchat_tumour <- computeCommunProb(cellchat_tumour)

In [ ]:
# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat_normal <- filterCommunication(cellchat_normal, min.cells = 10)
cellchat_tumour <- filterCommunication(cellchat_tumour, min.cells = 10)

In [ ]:
cellchat_normal <- computeCommunProbPathway(cellchat_normal)
cellchat_tumour <- computeCommunProbPathway(cellchat_tumour)

In [ ]:
cellchat_normal <- aggregateNet(cellchat_normal)
cellchat_tumour <- aggregateNet(cellchat_tumour)

In [ ]:
groupSize_normal <- as.numeric(table(cellchat_normal@idents))
groupSize_tumour <- as.numeric(table(cellchat_tumour@idents))

Warning message:
“UNRELIABLE VALUE: One of the ‘future.apply’ iterations (‘future_sapply-1’) unexpectedly generated random numbers without declaring so. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'future.seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'future.seed = NULL', or set option 'future.rng.onMisuse' to "ignore".”

In [ ]:
cellchat_normal <- netAnalysis_computeCentrality(cellchat_normal, slot.name = "netP")
cellchat_tumour <- netAnalysis_computeCentrality(cellchat_tumour, slot.name = "netP") 
# the slot 'netP' means the inferred intercellular communication network of signaling pathways

In [ ]:
saveRDS(cellchat_normal, "202401_cellchat_normal.rds")


In [ ]:
saveRDS(cellchat_tumour, "202401_cellchat_tumour.rds")


### try to split genomically normal and tumour out of tumour samples

length(str_imm_id_tumour$cell_id)

table(Numbat_and_Scitcem)

id_for_TT = c(rownames(subset(Numbat_and_Scitcem, Numbat_and_Scitcem == 'tumour_in_tumour')),
                                                    str_imm_id_tumour$cell_id)

id_for_TN = c(rownames(subset(Numbat_and_Scitcem, Numbat_and_Scitcem == 'normal_in_tumour')),
                                                    str_imm_id_tumour$cell_id)

cell_chat_tumour_tumour = subset_CellChat_single_dataset_rev(cellchat_tumour, cells.use = id_for_TT)

cell_chat_tumour_tumour

cell_chat_tumour_normal = subset_CellChat_single_dataset_rev(cellchat_tumour, cells.use = id_for_TN)

cell_chat_tumour_normal

cellchat_normal

dim(str_imm_id_tumour)

dim(str_imm_id_normal)

#### plot

In [ ]:
options(repr.plot.width=10, 
        repr.plot.height=10,
        repr.plot.res = 200)

netVisual_circle(cellchat_tumour@net$count, vertex.weight = groupSize_tumour, 
                 weight.scale = T, label.edge= F, title.name = "Number of interactions", 
                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
netVisual_circle(cellchat_normal@net$count, vertex.weight = groupSize_normal, 
                 weight.scale = T, label.edge= F, title.name = "Number of interactions", 
                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
mat <- cellchat_tumour@net$weight
par(mfrow = c(2,2), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  netVisual_circle(mat2, vertex.weight = groupSize_tumour, weight.scale = T, edge.weight.max = max(mat), 
                   title.name = rownames(mat)[i],
                   color.use = cell_type_order_with_Tumour_without_Monocytes_color)
}

In [ ]:
mat <- cellchat_normal@net$weight
par(mfrow = c(2,2), xpd=TRUE)
for (i in 1:nrow(mat)) {
  mat2 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat2[i, ] <- mat[i, ]
  netVisual_circle(mat2, vertex.weight = groupSize_normal, weight.scale = T, edge.weight.max = max(mat), 
                   title.name = rownames(mat)[i],
                color.use = cell_type_order_with_Tumour_without_Monocytes_color
) 
}

### pathway

This hierarchical plot consist of two components: 
the left portion shows autocrine and paracrine signaling to certain cell groups of interest (i.e, the defined vertex.receiver), 
and the right portion shows autocrine and paracrine signaling to the remaining cell groups in the dataset.

TGFb, WNT, ncWNT, BMP, KRAS

####  pathways that have significant communication

In [ ]:
sort(cellchat_normal@netP$pathways) # 92

In [ ]:
sort(cellchat_tumour@netP$pathways) # 100

In [ ]:
shared_sig_pathways = intersect(sort(cellchat_normal@netP$pathways), sort(cellchat_tumour@netP$pathways))

In [ ]:
shared_sig_pathways

In [ ]:
pathways.show <- 'BMP'

In [ ]:
length(levels(cellchat_tumour@idents))

In [ ]:
data.frame(levels(cellchat_tumour@idents), c(1:20)) #%>% 
#filter(str_detect(levels.cellchat_tumour.idents., 'genomically_normal'))

In [ ]:
# cell group index: all epi in NT
#NT_epi = c(6,7,9,10,17,18,20)
NT_epi = c(13,14,15,16,17,18,19,20)

In [ ]:
netVisual_aggregate(cellchat_tumour, signaling = pathways.show, 
                    vertex.receiver = NT_epi, layout = 'hierarchy')

netVisual_aggregate(cellchat_tumour, signaling = pathways.show, layout = "chord")
Error: not enough space for cells at track index '1'.

In [ ]:
options(repr.plot.width=7, 
        repr.plot.height=7,
        repr.plot.res = 200)

netVisual_heatmap(cellchat_tumour, color.heatmap = "Reds", measure = 'weight')

In [ ]:
options(repr.plot.width=7, 
        repr.plot.height=7,
        repr.plot.res = 200)

netVisual_heatmap(cellchat_normal, color.heatmap = "Reds")

In [ ]:
netVisual_heatmap(cellchat_normal, color.heatmap = "Reds", measure = 'weight')

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_heatmap(cellchat_tumour, signaling = pathways.show, color.heatmap = "Reds") 

In [ ]:
options(repr.plot.width=4, 
        repr.plot.height=4,
        repr.plot.res = 200)

netAnalysis_contribution(cellchat_tumour, signaling = pathways.show)

show all the significant interactions (L-R pairs) from some cell groups (defined by 'sources.use')
to other cell groups (defined by 'targets.use')
netVisual_bubble(cellchat, sources.use = 4, targets.use = c(5:11), remove.isolate = FALSE)

netVisual_bubble(cellchat_tumour, sources.use = 1, targets.use = NT_epi, 
                 signaling = c("BMP"), remove.isolate = FALSE)

pairLR.use <- extractEnrichedLR(cellchat, signaling = c("CCL","CXCL","FGF"))
netVisual_bubble(cellchat, sources.use = c(3,4), targets.use = c(5:8), 
                 pairLR.use = pairLR.use, remove.isolate = TRUE)

plotGeneExpression(cellchat, signaling = "CXCL")

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=5,
        repr.plot.res = 300)

netAnalysis_signalingRole_network(cellchat_tumour, signaling = pathways.show, 
                                  width = 25, height = 5, 
                                  font.size = 10) 

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netAnalysis_signalingRole_scatter(cellchat_tumour)

In [ ]:
options(repr.plot.width=6, 
        repr.plot.height=5,
        repr.plot.res = 200)

netAnalysis_signalingRole_scatter(cellchat_tumour, signaling = c("NOTCH"))

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=5,
        repr.plot.res = 300)

netAnalysis_signalingRole_network(cellchat_tumour, signaling = 'NOTCH', 
                                  width = 25, height = 5, 
                                  font.size = 10) 

In [ ]:
options(repr.plot.width=5, 
        repr.plot.height=6,
        repr.plot.res = 200)

netAnalysis_contribution(cellchat_tumour, signaling = 'NOTCH')

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(cellchat_tumour, pattern = "outgoing", 
                                  width = 30, height = 40, font.size = 10#,
                                  #signaling = shared_sig_pathways
                                 )

In [ ]:
options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(cellchat_tumour, pattern = "incoming", 
                                  width = 30, height = 40, font.size = 10#,
                                  #signaling = shared_sig_pathways
                                 )

options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(cellchat_normal, pattern = "outgoing", 
                                  width = 30, height = 40, font.size = 10)



options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(cellchat_normal, pattern = "incoming", 
                                  width = 30, height = 40, font.size = 10)


netVisual_bubble(cellchat_tumour, targets.use = NT_epi,  remove.isolate = TRUE, signaling = 'WNT')

### revise netAnalysis_signalingRole_heatmap to plot target cell type and pathways

# net: network communication prob of all ligand-receptor pair
# netP: of all pathway
# network centrality scores
centr_tumour = cellchat_tumour@netP$centr
centr_normal = cellchat_normal@netP$centr

outgoingt <- matrix(0, nrow = nlevels(cellchat_tumour@idents), ncol = length(centr_tumour))
incomingt <- matrix(0, nrow = nlevels(cellchat_tumour@idents), ncol = length(centr_tumour))

outgoingn <- matrix(0, nrow = nlevels(cellchat_normal@idents), ncol = length(centr_normal))
incomingn <- matrix(0, nrow = nlevels(cellchat_normal@idents), ncol = length(centr_normal))

dimnames(outgoingt) <- list(levels(cellchat_tumour@idents), names(centr_tumour))
dimnames(incomingt) <- dimnames(outgoingt)

dimnames(outgoingn) <- list(levels(cellchat_normal@idents), names(centr_normal))
dimnames(incomingn) <- dimnames(outgoingn)

for (i in 1:length(centr_tumour)) {
    outgoingt[,i] <- centr_tumour[[i]]$outdeg
    incomingt[,i] <- centr_tumour[[i]]$indeg
  }

for (i in 1:length(centr_normal)) {
    outgoingn[,i] <- centr_normal[[i]]$outdeg
    incomingn[,i] <- centr_normal[[i]]$indeg
  }

#### incoming signal

# epi 
g_TT_in = incomingt %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'tumour_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_tumour_in_tumour'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'TT')

g_NT_in = incomingt %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'genomically_normal')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_genomically_normal'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'NT')

g_NN_in = incomingn %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90)
     )+
labs(title = 'Normal sample')

options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_in + g_NT_in + plot_layout(guides = 'collect', ncol = 2) 

# imm + str 
g_NT_in_is = incomingt %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_normal_in_tumour'))) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'Tumour sample')

g_NN_in_is = incomingn %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
#filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     )+
labs(title = 'Normal sample')

options(repr.plot.width=14, 
        repr.plot.height=17,
        repr.plot.res = 300)

g_NT_in_is + g_NN_in_is + plot_layout(guides = 'collect', ncol = 2) 

options(repr.plot.width=16, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_in + g_NT_in + g_TT_in + g_NN_in_is + g_NT_in_is + plot_layout(guides = 'collect', ncol = 5) & 
theme(axis.title.y = element_blank())

# relative strength
# epi 
g_TT_in = sweep(t(incomingt), 1L, apply(t(incomingt), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'tumour_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_tumour_in_tumour'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'TT')

g_NT_in = sweep(t(incomingt), 1L, apply(t(incomingt), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'genomically_normal')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_genomically_normal'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'NT')

g_NN_in = sweep(t(incomingn), 1L, apply(t(incomingn), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90)
     )+
labs(title = 'Normal sample')

options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_in + g_NT_in + plot_layout(guides = 'collect', ncol =2)

# imm + str 
g_NT_in_is = sweep(t(incomingt), 1L, apply(t(incomingt), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_normal_in_tumour'))) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'Tumour sample')

g_NN_in_is = sweep(t(incomingn), 1L, apply(t(incomingn), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
#filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     )+
labs(title = 'Normal sample')

options(repr.plot.width=14, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_in_is + g_NT_in_is + plot_layout(guides = 'collect')

options(repr.plot.width=16, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_in + g_NT_in + g_TT_in + g_NN_in_is + g_NT_in_is + plot_layout(guides = 'collect', ncol = 5) & 
theme(axis.title.y = element_blank())

#### outgoing signal

# epi 
g_TT_out = outgoingt %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'tumour_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_tumour_in_tumour'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'TT')

g_NT_out = outgoingt %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'genomically_normal')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_genomically_normal'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'NT')

g_NN_out = outgoingn %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90)
     )+
labs(title = 'Normal sample')


# imm + str 
g_NT_out_is = outgoingt %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_normal_in_tumour'))) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'Tumour sample')

g_NN_out_is = outgoingn %>% t() %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
#filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,4))+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     )+
labs(title = 'Normal sample')

options(repr.plot.width=16, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_out + g_NT_out + g_TT_out + g_NN_out_is + g_NT_out_is + plot_layout(guides = 'collect', ncol = 5) & 
theme(axis.title.y = element_blank())

# relative strength
# epi 
g_TT_out = sweep(t(outgoingt), 1L, apply(t(outgoingt), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'tumour_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_tumour_in_tumour'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'TT')

g_NT_out = sweep(t(outgoingt), 1L, apply(t(outgoingt), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
filter(str_detect(Celltype, 'genomically_normal')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_genomically_normal'))) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'NT')

g_NN_out = sweep(t(outgoingn), 1L, apply(t(incomingn), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90)
     )+
labs(title = 'Normal sample')

# imm + str 
g_NT_out_is = sweep(t(outgoingt), 1L, apply(t(outgoingt), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = paste0(epi_cell_order,'_normal_in_tumour'))) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     ) +
labs(title = 'Tumour sample')

g_NN_out_is = sweep(t(outgoingn), 1L, apply(t(outgoingn), 1, max), '/', check.margin = FALSE) %>%
data.frame() %>% 
rownames_to_column('Pathway') %>%
pivot_longer(!Pathway, names_to = 'Celltype', values_to = 'Centrality') %>%
#filter(str_detect(Celltype, 'normal_in_tumour')) %>%
#filter(Celltype %in% epi_cell_order) %>%
filter(Pathway %in% shared_sig_pathways) %>%
#mutate(Celltype = factor(Celltype, levels = epi_cell_order)) %>% 
filter(Celltype %in% imm_str_cell_order) %>%
mutate(Celltype = factor(Celltype, levels = imm_str_cell_order)) %>% 
ggplot() +
geom_tile(aes(x = Celltype, y = Pathway, fill = Centrality)) +
#scale_fill_gradient2(low = 'white', mid = 'yellow', high = 'green', midpoint = 0.5, name = 'Relative strength') +
scale_fill_gradientn(colors=c('white', met.brewer("VanGogh3")), limits = c(0,1), name = 'Relative strength')+
plot_theme_tmod +
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      #rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90),
      axis.title.y = element_blank(),
      axis.text.y = element_blank()
     )+
labs(title = 'Normal sample')

options(repr.plot.width=16, 
        repr.plot.height=20,
        repr.plot.res = 300)

g_NN_out + g_NT_out + g_TT_out + g_NN_out_is + g_NT_out_is + plot_layout(guides = 'collect', ncol = 5) & 
theme(axis.title.y = element_blank())

### try subset if the centrality will change > YES

cellchat_normal_in_tumour = subset_CellChat_single_dataset_rev(cellchat_tumour, 
                                   idents.use = c(paste0(epi_cell_order, '_normal_in_tumour'), 
                                                  imm_str_cell_order))

shared_sig_pathways_NT = intersect(sort(cellchat_normal@netP$pathways), 
                                   sort(cellchat_normal_in_tumour@netP$pathways))
length(shared_sig_pathways_NT)

centr_NT = cellchat_normal_in_tumour@netP$centr

outgoingNT <- matrix(0, nrow = nlevels(cellchat_normal_in_tumour@idents), ncol = length(centr_NT))
incomingNT <- matrix(0, nrow = nlevels(cellchat_normal_in_tumour@idents), ncol = length(centr_NT))

dimnames(outgoingNT) <- list(levels(cellchat_normal_in_tumour@idents), names(centr_NT))
dimnames(incomingNT) <- dimnames(outgoinNT)

for (i in 1:length(centr_NT)) {
    outgoingNT[,i] <- centr_NT[[i]]$outdeg
    incomingNT[,i] <- centr_NT[[i]]$indeg

}

#### check some pathways

pathways.show = 'WNT'

options(repr.plot.width=7, 
        repr.plot.height=7,
        repr.plot.res = 200)


netVisual_heatmap(cellchat_normal_in_tumour, signaling = pathways.show, color.heatmap = "Reds") 

options(repr.plot.width=5, 
        repr.plot.height=5,
        repr.plot.res = 200)


netAnalysis_contribution(cellchat_normal_in_tumour, signaling = pathways.show)


options(repr.plot.width=8, 
        repr.plot.height=5,
        repr.plot.res = 200)

netAnalysis_signalingRole_network(cellchat_normal_in_tumour, signaling = pathways.show, 
                                  width = 14, height = 5, 
                                  font.size = 10) 

#### play with subgraph to keep the centrality?

G <- igraph::graph_from_adjacency_matrix(cellchat_tumour@netP$prob[,,'MK'], mode = "directed", weighted = T)

group.existing.index

V(G)[group.existing.index]$name

E(G)[.from(26)]
only Stem (26) and Tuft (45) have edges

options(repr.plot.width=18, 
        repr.plot.height=15,
        repr.plot.res = 300)

plot(G, layout = layout_with_kk, vertex.size = 10, edge.arrow.size = 0.5, edge.arrow.width = 0.5)

G2 = induced_subgraph(G, group.existing.index, impl = 'copy_and_delete')

G4 = induced_subgraph(G, group.existing.index, impl = 'create_from_scratch')

plot(G2, layout = layout_with_kk, vertex.size = 10, edge.arrow.size = 0.5, edge.arrow.width = 0.5)

G3 = igraph::graph_from_adjacency_matrix(cellchat_normal_in_tumour@netP$prob[,,'MK'], mode = "directed", weighted = T)

plot(G3, layout = layout_with_kk, vertex.size = 10, edge.arrow.size = 0.5, edge.arrow.width = 0.5)

strength(G, mode = 'out') 

strength(G2, mode = 'out') 

strength(G3, mode = 'out') 

head(cellchat_tumour@netP$centr$MK)

head(cellchat_normal_in_tumour@netP$centr$MK)

#### outgoing

selectK(cellchat, pattern = "outgoing")

nPatterns = 3
cellchat <- identifyCommunicationPatterns(cellchat, pattern = "outgoing", k = nPatterns)

netAnalysis_river(cellchat, pattern = "outgoing")

netAnalysis_dot(cellchat, pattern = "outgoing")

#### incoming


selectK(cellchat, pattern = "incoming")

nPatterns = 3
cellchat <- identifyCommunicationPatterns(cellchat, pattern = "incoming", k = nPatterns)

netAnalysis_river(cellchat, pattern = "incoming")

netAnalysis_dot(cellchat, pattern = "incoming")

### similarity between signaling pathways and group them

#### functional

cellchat <- computeNetSimilarity(cellchat, type = "functional")
cellchat <- netEmbedding(cellchat, type = "functional")

cellchat <- netClustering(cellchat, type = "functional")

netVisual_embedding(cellchat, type = "functional", label.size = 3.5)

netVisual_embeddingZoomIn(cellchat, type = "functional", nCol = 2)

#### structural

cellchat <- computeNetSimilarity(cellchat, type = "structural")
cellchat <- netEmbedding(cellchat, type = "structural")

cellchat <- netClustering(cellchat, type = "structural")

netVisual_embedding(cellchat, type = "structural", label.size = 3.5)

netVisual_embeddingZoomIn(cellchat, type = "structural", nCol = 2)

### merge and compare tumour sample vs normal sample

net$prob and net$pval updated the Idents in the background

colnames(cellchat_normal_in_tumour@net$count) = colnames(cellchat_normal_in_tumour@net$prob) 
colnames(cellchat_normal_in_tumour@net$weight) = colnames(cellchat_normal_in_tumour@net$prob) 
colnames(cellchat_normal_in_tumour@netP$prob) = colnames(cellchat_normal_in_tumour@net$prob) 

rownames(cellchat_normal_in_tumour@net$count) = colnames(cellchat_normal_in_tumour@net$prob) 
rownames(cellchat_normal_in_tumour@net$weight) = colnames(cellchat_normal_in_tumour@net$prob) 
rownames(cellchat_normal_in_tumour@netP$prob) = colnames(cellchat_normal_in_tumour@net$prob) 

cellchat_normal_test = cellchat_normal
cellchat_tumour_test = cellchat_tumour

names(cellchat_normal_test@net)

names(cellchat_normal_test@netP)

#### modify normal sample obj: net and netP

#@net is used in compareInteractions, rankNet, etc
#for normal samples: remove Monocytes and add Tumour > rename Monocytes to Tumour (13)
#count and weight all 0
sum(cellchat_normal_test@net$count[13,])
sum(cellchat_normal_test@net$count[,13])

sum(cellchat_normal_test@net$weight[13,])
sum(cellchat_normal_test@net$weight[,13])

rownames(cellchat_normal_test@net$count)[13] = 'Tumour'
colnames(cellchat_normal_test@net$count)[13] = 'Tumour'

rownames(cellchat_normal_test@net$weight)[13] = 'Tumour'
colnames(cellchat_normal_test@net$weight)[13] = 'Tumour'

#reorder cell type
cellchat_normal_test@net$count = cellchat_normal_test@net$count[order(colnames(cellchat_normal_test@net$count)),
                                                                order(colnames(cellchat_normal_test@net$count))]

cellchat_normal_test@net$weight = cellchat_normal_test@net$weight[order(colnames(cellchat_normal_test@net$weight)),
                                                                  order(colnames(cellchat_normal_test@net$weight))]

#prob is a 3d array: cell type as rows (20) x col (20) x LRpairs (1619)
#all prob = 0
#rename row

sum(cellchat_normal_test@net$prob[13,13,])

rownames(cellchat_normal_test@net$prob)[13]  = 'Tumour'
colnames(cellchat_normal_test@net$prob)[13]  = 'Tumour'

#pval is also a 3d array
#why are there pvals != 1.....
#anyway put all as 1
sum(cellchat_normal_test@net$pval[13,13,])


rownames(cellchat_normal_test@net$pval)[13]  = 'Tumour'
colnames(cellchat_normal_test@net$pval)[13]  = 'Tumour'

cellchat_normal_test@net$pval[13,13,] = 1

sum(cellchat_normal_test@netP$prob[13,13,])

cellchat_normal_test@net$prob = cellchat_normal_test@net$prob[order(colnames(cellchat_normal_test@net$prob)),
                                                              order(colnames(cellchat_normal_test@net$prob)),]

cellchat_normal_test@net$pval = cellchat_normal_test@net$pval[order(colnames(cellchat_normal_test@net$pval)),
                                                              order(colnames(cellchat_normal_test@net$pval)),]

#### modify tumour sample obj

#for tumour samples: remove Monocytes
cellchat_tumour_test@net$count = cellchat_tumour_test@net$count[-c(13), -c(13)]
cellchat_tumour_test@net$weight = cellchat_tumour_test@net$weight[-c(13), -c(13)]

cellchat_tumour_test@net$prob = cellchat_tumour_test@net$prob[-c(13), -c(13),]
cellchat_tumour_test@net$pval = cellchat_tumour_test@net$pval[-c(13), -c(13),]

### merge objs

In [ ]:
# merge
object.list <- list(Normal = cellchat_normal, 
                    Tumour = cellchat_tumour#,
                    #Normal_in_Tumour = cellchat_normal_in_tumour
                    #Tumour_in_Tumour = cell_chat_tumour_tumour
                   )

cellchat <- mergeCellChat(object.list, add.names = names(object.list), cell.prefix = FALSE)
# if use TT
# Error in mergeCellChat(object.list, add.names = names(object.list)): 
# Duplicated cell names were detected across datasets!! Please set cell.prefix = TRUE

# Merge the following slots: 'data.signaling','images','net', 'netP','meta', 
#'idents', 'var.features' , 'DB', and 'LR'.

In [ ]:
cellchat #977 genes, 70553 cells

In [ ]:
cellchat@idents$joint = factor(cellchat@idents$joint, 
                               levels = cell_type_order_with_Tumour_without_Monocytes)
cellchat@idents$Normal = factor(cellchat@idents$Normal, 
                               levels = cell_type_order_with_Tumour_without_Monocytes)
cellchat@idents$Tumour = factor(cellchat@idents$Tumour, 
                               levels = cell_type_order_with_Tumour_without_Monocytes)


#cellchat@idents$Normal_in_Tumour = factor(cellchat@idents$Normal_in_Tumour, 
#                               levels = cell_type_order)
#cellchat@idents$Tumour_in_Tumour = factor(cellchat@idents$Tumour_in_Tumour, 
#                               levels = cell_type_order)

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

gg1 <- compareInteractions(cellchat, show.legend = F, group = c(1,2), color.use = c('#808080','#ff7f0e'),
                           digits = 2, size.text = 18)

gg2 <- compareInteractions(cellchat, show.legend = F, group = c(1,2), measure = "weight",
                           color.use = c('#808080','#ff7f0e'),
                           digits = 2, size.text = 18)
gg1 + gg2

too much
netVisual_diffInteraction(cellchat, weight.scale = T)

netVisual_diffInteraction(cellchat, weight.scale = T, measure = "weight")

In [ ]:
colnames(cellchat@net[['Tumour']][['count']])

In [ ]:
colnames(cellchat@net[['Normal']][['count']])

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_heatmap(cellchat, width = 16, font.size.title = 15, font.size = 12,
                  color.use = cell_type_order_with_Tumour_without_Monocytes_color
)
#some issue to plot togeter
#ERROR while rich displaying an object: 
#Error in grid.Call.graphics(C_downvppath, name$path, name$name, strict): 
#Viewport 'Relative values_heatmap_body_1_1' was not found

#Error in obj2 - obj1: non-conformable arrays
#Traceback:
#
#1. netVisual_heatmap(cellchat, width = 16, font.size.title = 15, 
# .     font.size = 12)

In [ ]:
options(repr.plot.width=8, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_heatmap(cellchat, measure = "weight", width = 16, font.size.title = 15, font.size = 12,
                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
options(repr.plot.width=16, 
        repr.plot.height=8,
        repr.plot.res = 200)

weight.max <- getMaxWeight(object.list, attribute = c("idents","count"))

par(mfrow = c(1,2), xpd=TRUE)

for (i in 1:length(object.list)) {
  netVisual_circle(object.list[[i]]@net$count, weight.scale = T, label.edge= F, 
                   edge.weight.max = weight.max[2], edge.width.max = 12, 
                   #title.name = paste0("Number of interactions - ", names(object.list)[i]),
                  color.use = cell_type_order_with_Tumour_without_Monocytes_color)
}

In [ ]:
# need centr
options(repr.plot.width=16, 
        repr.plot.height=8,
        repr.plot.res = 200)

num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})

#weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets

gg <- list()
for (i in 1:length(object.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], #title = names(object.list)[i], 
                                               font.size.title = 18, font.size = 15, label.size = 5,
                                               color.use = cell_type_order_with_Tumour_without_Monocytes_color
                                               #,weight.MinMax = weight.MinMax
                                              ) + 
    xlim(0, 20) + 
    ylim(0, 12)
}

patchwork::wrap_plots(plots = gg)

In [ ]:
# outgoing and incoming per pathway 


#### similarity

Functional similarity: High degree of functional similarity indicates major senders and receivers are similar, and it can be interpreted as the two signaling pathways or two ligand-receptor pairs exhibit similar and/or redundant roles.

In [ ]:
cellchat <- computeNetSimilarityPairwise(cellchat, type = "functional")
#> Compute signaling network similarity for datasets 1 2
cellchat <- netEmbedding(cellchat, type = "functional")
#> Manifold learning of the signaling networks for datasets 1 2
cellchat <- netClustering(cellchat, type = "functional")
#> Classification learning of the signaling networks for datasets 1 2

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_embeddingPairwise(cellchat, type = "functional", label.size = 3)

Structural similarity: A structural similarity was used to compare their signaling network structure, without considering the similarity of senders and receivers. NB: Structural similarity analysis is applicable to multiple datsets with the same cell type composition or the vastly different cell type composition.

In [ ]:
cellchat <- computeNetSimilarityPairwise(cellchat, type = "structural")
#> Compute signaling network similarity for datasets 1 2
cellchat <- netEmbedding(cellchat, type = "structural")
#> Manifold learning of the signaling networks for datasets 1 2
cellchat <- netClustering(cellchat, type = "structural")
#> Classification learning of the signaling networks for datasets 1 2

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=8,
        repr.plot.res = 200)

netVisual_embeddingPairwise(cellchat, type = "structural", label.size = 3.5)

In [ ]:
rankSimilarity(cellchat, type = "functional")

In [ ]:
rankSimilarity(cellchat, type = "structural")

#### information flows (sum of communication probability among all pairs == total weights)
- conserved and context-specifc signalling pathways 

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=16,
        repr.plot.res = 200)

gg1 <- rankNet(cellchat, mode = "comparison", stacked = T, do.stat = TRUE, font.size = 10, measure = "count",
              color.use = c('#808080', '#ff7f0e'))
gg2 <- rankNet(cellchat, mode = "comparison", stacked = F, do.stat = TRUE, font.size = 10, measure = "count",
              color.use = c('#808080', '#ff7f0e'))
gg1 + gg2

In [ ]:
gg3 <- rankNet(cellchat, mode = "comparison", stacked = T, do.stat = TRUE, font.size = 10, measure = "weight",
              color.use = c('#808080', '#ff7f0e'))
gg4 <- rankNet(cellchat, mode = "comparison", stacked = F, do.stat = TRUE, font.size = 10, measure = "weight",
              color.use = c('#808080', '#ff7f0e'))
gg3 + gg4

### pathways vs cell groups

In [ ]:
i = 1
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=16,
        repr.plot.res = 200)

netAnalysis_signalingRole_heatmap(object.list[[1]], pattern = "all", 
                                        signaling = pathway.union, title = names(object.list)[1],  
                                  width = 20, height = 35, color.heatmap = "OrRd", font.size = 10,
                                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)

In [ ]:
netAnalysis_signalingRole_heatmap(object.list[[2]], pattern = "all", 
                                        signaling = pathway.union, 
                                  title = names(object.list)[2], 
                                  width = 20, height = 35, color.heatmap = "OrRd", font.size = 10,
                                  color.use = cell_type_order_with_Tumour_without_Monocytes_color)

netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "outgoing", 
                                        signaling = pathway.union, 
                                        title = names(object.list)[i+1], width = 20, height = 35, font.size = 10)

### LR pairs from some cell groups to some cell groups (differences in community prob)

In [ ]:
length(levels(cellchat@idents$joint))

In [ ]:
shared_sig_pathways = intersect(sort(cellchat@netP$Normal$pathways), sort(cellchat@netP$Tumour$pathways))

In [ ]:
(shared_sig_pathways) # 88

In [ ]:
data.frame(levels(cellchat@idents$joint), c(1:20))

In [ ]:
# from Tumour to everything
options(repr.plot.width=12, 
        repr.plot.height=25,
        repr.plot.res = 200)

netVisual_bubble(cellchat, sources.use = 20, targets.use = c(1:20),  comparison = c(1, 2), angle.x = 45)

In [ ]:
# from Fibroblast to Stem/Tumour (the targets.use doesnt let me do one cell group...)
options(repr.plot.width=12, 
        repr.plot.height=20,
        repr.plot.res = 200)

netVisual_bubble(cellchat, sources.use = 11, targets.use = c(13,20),  comparison = c(1, 2), angle.x = 45)

### differential at pathway level

In [ ]:
# plot all the significant pathways
options(repr.plot.width=12, 
        repr.plot.height=6,
        repr.plot.res = 200)

pdf(paste0(new_data_folder,'/20231129_cellchat_pathway_cell_groups_signals.pdf'), paper = 'a4r')

for(i in shared_sig_pathways){
    pathways.show = i
    
    weight.max <- getMaxWeight(object.list, slot.name = c("netP"), attribute = pathways.show) 
    # control the edge weights across different datasets
    
    par(mfrow = c(1,2), xpd=TRUE)
    
    for (i in 1:length(object.list)) {
        netVisual_aggregate(object.list[[i]], signaling = pathways.show, layout = "circle", 
                            edge.weight.max = weight.max[1], edge.width.max = 10, 
                            signaling.name = paste(pathways.show, names(object.list)[i]),
                            color.use = cell_type_order_with_Tumour_without_Monocytes_color)
    }
}
dev.off()

### identify dysfunctional signaling by using differential expression

In [ ]:
# define a positive dataset, i.e., the dataset with positive fold change against the other dataset
pos.dataset = "Tumour"
# define a char name used for storing the results of differential expression analysis
features.name = pos.dataset
# perform differential expression analysis
cellchat <- identifyOverExpressedGenes(cellchat, group.dataset = "datasets", 
                                       pos.dataset = pos.dataset, 
                                       features.name = features.name, only.pos = FALSE, 
                                       thresh.pc = 0.1, thresh.fc = 0.1, thresh.p = 1)

#> Use the joint cell labels from the merged CellChat object
# map the results of differential expression analysis onto the inferred cell-cell communications to easily manage/subset the ligand-receptor pairs of interest
net <- netMappingDEG(cellchat, features.name = features.name)
# extract the ligand-receptor pairs with upregulated ligands 
net.up <- subsetCommunication(cellchat, net = net, datasets = "Tumour",
                              ligand.logFC = 0.2, receptor.logFC = NULL)

net.down <- subsetCommunication(cellchat, net = net, datasets = "Normal",
                                ligand.logFC = -0.2, receptor.logFC = NULL)



In [ ]:
saveRDS(cellchat, "202401_cellchat.rds")


In [ ]:
# keep only expressed (not NA) receptors
net.up = net.up[which(net.up$receptor.logFC != 'NA'),] #1855 

In [ ]:
net.down = net.down[which(net.down$receptor.logFC != 'NA'),] #411

In [ ]:
gene.up <- extractGeneSubsetFromPair(net.up, cellchat)
gene.down <- extractGeneSubsetFromPair(net.down, cellchat)

In [ ]:
pairLR.use.up = net.up[, "interaction_name", drop = F]
pairLR.use.down = net.down[, "interaction_name", drop = F]

In [ ]:
options(repr.plot.width=12, 
        repr.plot.height=20,
        repr.plot.res = 200)
#' @param remove.isolate whether remove the entire empty column, i.e., communication between certain cell groups
gg1 <- netVisual_bubble(cellchat, pairLR.use = pairLR.use.up, sources.use = 11, targets.use = 13, 
                        comparison = c(1, 2),  angle.x = 90, remove.isolate = T,
                        title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))

gg2 <- netVisual_bubble(cellchat, pairLR.use = pairLR.use.down, sources.use = 11, targets.use = 13, 
                        comparison = c(1, 2),  angle.x = 90, remove.isolate = T,
                        title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))

gg1 + gg2

In [ ]:
# Chord diagram
options(repr.plot.width=10, 
        repr.plot.height=10,
        repr.plot.res = 200)

netVisual_chord_gene(object.list[[2]], sources.use = 11, targets.use = 13, 
                     slot.name = 'net', net = net.up, lab.cex = 0.8, small.gap = 3.5, 
                     title.name = paste0("Up-regulated signaling in ", names(object.list)[2]))

In [ ]:
netVisual_chord_gene(object.list[[1]], sources.use = 11, targets.use = 13, 
                     slot.name = 'net', net = net.down, lab.cex = 0.8, small.gap = 3.5, 
                     title.name = paste0("Down-regulated signaling in ", names(object.list)[2]))

#### pathway

In [ ]:
pathways.show <- c("WNT") 
ht <- list()
for (i in 1:length(object.list)) {
  ht[[i]] <- netVisual_heatmap(object.list[[i]], signaling = pathways.show, 
                               color.heatmap = "Reds",
                               title.name = paste(pathways.show, "signaling ",names(object.list)[i]),
                              ,
                                 color.use = cell_type_order_with_Tumour_without_Monocytes_color)
}

In [ ]:
options(repr.plot.width=6, 
        repr.plot.height=6,
        repr.plot.res = 200)
ht[[1]]

In [ ]:
ht[[2]]

In [ ]:
## Shut down parallel workers
future::plan("sequential")

### Immune cell annotations

anno_folder = '/fast/users/twei_m/work/crc/datasets/anno/CRC/3p/'

immune_folder = '/fast/users/twei_m/work/immune/seurat_objects/'

### anno folder

#### an annotation file

singlecell_CRC_patients_3prime_annotation = read.csv(paste0(anno_folder,
                                                            'singlecell_CRC_patients_3prime_annotation.csv'))

colnames(singlecell_CRC_patients_3prime_annotation)[1] = 'cell_id'

singlecell_CRC_patients_3prime_annotation

#### original Florian's object: Disaster

seu_all_final = readRDS(paste0(anno_folder,'seu_all_final.rds'))

#### # wtf one cell with anno both in imm and str


# empty: 'cell_type_imm_custom','cell_type_imm_simple'
seu_all_final@meta.data[c('cell_type_epi','cell_type_imm','cell_type_str')]

table(seu_all_final@meta.data[c('cell_type_str', 'cell_type_imm')])

#### Philips latest object

immune_philips = readRDS(paste0(immune_folder,'immune_annotated.RDS'))

str_philips = readRDS(paste0(immune_folder,'stromal_annotated.RDS'))

head(immune_philips@meta.data)

table(immune_philips@meta.data[c('cell_type', 'cell_type2')])

table(str_philips@meta.data[c('cell_type', 'cell_type2')])

# if double annotated
table(rownames(str_philips@meta.data) %in% rownames(immune_philips@meta.data))

### ImSigGenes_immunetumor.xlsx